In [1]:
# import some library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import matplotlib.pyplot as plt
import itertools
import time
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV

from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam, Adamax, Adagrad
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)
set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# read data
with open('../data/HCD35_pos.pickle', mode='rb') as fp:
    df_3 = pickle.load(fp)
    
df = pd.concat([
    df_3,
    pd.read_csv('../data/decomp_pos.csv').drop('Subclass', axis=1)
], axis=1)

# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)
features = df.drop('Subclass', axis=1)

# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [3]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [4]:
def create_model(optimizer='rmsprop'):
    tf.set_random_seed(42)
    c=0.4
    # make keras model
    start = time.time()
    inputs = Input(shape=(X_train.shape[1],))
    x = Dense(1024, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(c)(x)
    predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
    model = Model(inputs=inputs, outputs=predictions)
    
    model.compile(
        loss='categorical_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy']
    )
    
    return model

In [5]:
model = KerasClassifier(
    build_fn=create_model, 
    verbose=2, 
    epochs=100, 
    batch_size=1000
)

In [6]:
p = [
        'SGD', 'RMSprop', 'Adagrad', 'Adadelta', 
        'Adam', 'Adamax', 'Nadam'
]  # optimizers

params = dict(
    optimizer=p
             )

In [7]:
grid = GridSearchCV(
    estimator=model, 
    param_grid=params
)

In [8]:
grid_result = grid.fit(X_train, y_train_for_keras)

Epoch 1/100
 - 11s - loss: 1.8890 - acc: 0.2710
Epoch 2/100
 - 2s - loss: 1.9172 - acc: 0.2486
Epoch 3/100
 - 5s - loss: 1.7932 - acc: 0.2777
Epoch 4/100
 - 3s - loss: 1.8032 - acc: 0.2598
Epoch 5/100
 - 2s - loss: 1.7018 - acc: 0.3035
Epoch 6/100
 - 2s - loss: 1.7198 - acc: 0.3124
Epoch 7/100
 - 3s - loss: 1.6855 - acc: 0.3102
Epoch 8/100
 - 2s - loss: 1.6660 - acc: 0.2811
Epoch 9/100
 - 2s - loss: 1.6620 - acc: 0.3091
Epoch 10/100
 - 2s - loss: 1.6587 - acc: 0.2822
Epoch 11/100
 - 3s - loss: 1.5653 - acc: 0.3068
Epoch 12/100
 - 3s - loss: 1.6080 - acc: 0.3024
Epoch 13/100
 - 2s - loss: 1.5823 - acc: 0.3091
Epoch 14/100
 - 3s - loss: 1.5469 - acc: 0.3124
Epoch 15/100
 - 2s - loss: 1.5606 - acc: 0.3080
Epoch 16/100
 - 3s - loss: 1.4907 - acc: 0.3303
Epoch 17/100
 - 7s - loss: 1.6055 - acc: 0.3371
Epoch 18/100
 - 15s - loss: 1.5171 - acc: 0.3393
Epoch 19/100
 - 11s - loss: 1.4825 - acc: 0.3516
Epoch 20/100
 - 3s - loss: 1.4927 - acc: 0.3326
Epoch 21/100
 - 2s - loss: 1.4553 - acc: 0.334

Epoch 72/100
 - 1s - loss: 1.2206 - acc: 0.4580
Epoch 73/100
 - 1s - loss: 1.2411 - acc: 0.4502
Epoch 74/100
 - 1s - loss: 1.1399 - acc: 0.4871
Epoch 75/100
 - 1s - loss: 1.1880 - acc: 0.4849
Epoch 76/100
 - 1s - loss: 1.2101 - acc: 0.4614
Epoch 77/100
 - 1s - loss: 1.2338 - acc: 0.4244
Epoch 78/100
 - 1s - loss: 1.2045 - acc: 0.4602
Epoch 79/100
 - 1s - loss: 1.2002 - acc: 0.4625
Epoch 80/100
 - 1s - loss: 1.1478 - acc: 0.4793
Epoch 81/100
 - 1s - loss: 1.1643 - acc: 0.4726
Epoch 82/100
 - 1s - loss: 1.1957 - acc: 0.4681
Epoch 83/100
 - 1s - loss: 1.1781 - acc: 0.4815
Epoch 84/100
 - 1s - loss: 1.1616 - acc: 0.4860
Epoch 85/100
 - 1s - loss: 1.1467 - acc: 0.4905
Epoch 86/100
 - 1s - loss: 1.1773 - acc: 0.4714
Epoch 87/100
 - 1s - loss: 1.1808 - acc: 0.4826
Epoch 88/100
 - 1s - loss: 1.1457 - acc: 0.4894
Epoch 89/100
 - 1s - loss: 1.1655 - acc: 0.4983
Epoch 90/100
 - 1s - loss: 1.1506 - acc: 0.5039
Epoch 91/100
 - 1s - loss: 1.1636 - acc: 0.4871
Epoch 92/100
 - 1s - loss: 1.1458 - acc:

 - 2s - loss: 0.3786 - acc: 0.8578
Epoch 44/100
 - 1s - loss: 0.4200 - acc: 0.8623
Epoch 45/100
 - 2s - loss: 0.2332 - acc: 0.9115
Epoch 46/100
 - 2s - loss: 0.2250 - acc: 0.9216
Epoch 47/100
 - 1s - loss: 0.2237 - acc: 0.9339
Epoch 48/100
 - 1s - loss: 0.1867 - acc: 0.9395
Epoch 49/100
 - 1s - loss: 0.1361 - acc: 0.9451
Epoch 50/100
 - 1s - loss: 0.1507 - acc: 0.9384
Epoch 51/100
 - 2s - loss: 0.1617 - acc: 0.9351
Epoch 52/100
 - 2s - loss: 0.1951 - acc: 0.9272
Epoch 53/100
 - 2s - loss: 0.2327 - acc: 0.9082
Epoch 54/100
 - 1s - loss: 0.3302 - acc: 0.8981
Epoch 55/100
 - 2s - loss: 0.1887 - acc: 0.9227
Epoch 56/100
 - 2s - loss: 0.1715 - acc: 0.9373
Epoch 57/100
 - 2s - loss: 0.1397 - acc: 0.9474
Epoch 58/100
 - 1s - loss: 0.2516 - acc: 0.9250
Epoch 59/100
 - 2s - loss: 0.2861 - acc: 0.9003
Epoch 60/100
 - 1s - loss: 0.1862 - acc: 0.9295
Epoch 61/100
 - 1s - loss: 0.1570 - acc: 0.9440
Epoch 62/100
 - 1s - loss: 0.1915 - acc: 0.9317
Epoch 63/100
 - 2s - loss: 0.2331 - acc: 0.9306
Epoch

Epoch 15/100
 - 2s - loss: 0.4767 - acc: 0.8087
Epoch 16/100
 - 2s - loss: 0.4386 - acc: 0.8333
Epoch 17/100
 - 2s - loss: 0.4359 - acc: 0.8210
Epoch 18/100
 - 2s - loss: 0.6589 - acc: 0.7964
Epoch 19/100
 - 2s - loss: 1.0755 - acc: 0.7148
Epoch 20/100
 - 2s - loss: 0.6158 - acc: 0.7595
Epoch 21/100
 - 2s - loss: 0.6321 - acc: 0.7685
Epoch 22/100
 - 2s - loss: 0.4955 - acc: 0.8210
Epoch 23/100
 - 2s - loss: 0.4479 - acc: 0.8423
Epoch 24/100
 - 2s - loss: 0.3047 - acc: 0.8949
Epoch 25/100
 - 2s - loss: 0.2937 - acc: 0.8881
Epoch 26/100
 - 2s - loss: 0.3215 - acc: 0.8814
Epoch 27/100
 - 2s - loss: 0.3664 - acc: 0.8647
Epoch 28/100
 - 2s - loss: 0.4703 - acc: 0.8423
Epoch 29/100
 - 2s - loss: 0.4361 - acc: 0.8479
Epoch 30/100
 - 1s - loss: 0.4526 - acc: 0.8456
Epoch 31/100
 - 1s - loss: 0.2633 - acc: 0.9072
Epoch 32/100
 - 1s - loss: 0.2919 - acc: 0.9038
Epoch 33/100
 - 2s - loss: 0.2204 - acc: 0.9195
Epoch 34/100
 - 2s - loss: 0.2381 - acc: 0.9217
Epoch 35/100
 - 2s - loss: 0.2012 - acc:

Epoch 86/100
 - 5s - loss: 0.0934 - acc: 0.9686
Epoch 87/100
 - 4s - loss: 0.0952 - acc: 0.9686
Epoch 88/100
 - 4s - loss: 0.0743 - acc: 0.9731
Epoch 89/100
 - 5s - loss: 0.0762 - acc: 0.9709
Epoch 90/100
 - 4s - loss: 0.0782 - acc: 0.9720
Epoch 91/100
 - 5s - loss: 0.0769 - acc: 0.9664
Epoch 92/100
 - 3s - loss: 0.0897 - acc: 0.9664
Epoch 93/100
 - 5s - loss: 0.0760 - acc: 0.9709
Epoch 94/100
 - 3s - loss: 0.0889 - acc: 0.9686
Epoch 95/100
 - 3s - loss: 0.0702 - acc: 0.9765
Epoch 96/100
 - 5s - loss: 0.0707 - acc: 0.9742
Epoch 97/100
 - 3s - loss: 0.0797 - acc: 0.9742
Epoch 98/100
 - 3s - loss: 0.0819 - acc: 0.9742
Epoch 99/100
 - 6s - loss: 0.0880 - acc: 0.9630
Epoch 100/100
 - 3s - loss: 0.0783 - acc: 0.9731
Epoch 1/100
 - 9s - loss: 1.8494 - acc: 0.2542
Epoch 2/100
 - 4s - loss: 3.1443 - acc: 0.2822
Epoch 3/100
 - 3s - loss: 6.0053 - acc: 0.2587
Epoch 4/100
 - 3s - loss: 3.2973 - acc: 0.2766
Epoch 5/100
 - 3s - loss: 2.9837 - acc: 0.2755
Epoch 6/100
 - 4s - loss: 1.6895 - acc: 0.44

Epoch 57/100
 - 1s - loss: 0.1281 - acc: 0.9575
Epoch 58/100
 - 1s - loss: 0.1176 - acc: 0.9664
Epoch 59/100
 - 1s - loss: 0.1128 - acc: 0.9653
Epoch 60/100
 - 1s - loss: 0.1100 - acc: 0.9609
Epoch 61/100
 - 1s - loss: 0.1111 - acc: 0.9620
Epoch 62/100
 - 1s - loss: 0.1245 - acc: 0.9664
Epoch 63/100
 - 1s - loss: 0.1132 - acc: 0.9653
Epoch 64/100
 - 1s - loss: 0.1284 - acc: 0.9664
Epoch 65/100
 - 1s - loss: 0.1077 - acc: 0.9687
Epoch 66/100
 - 1s - loss: 0.1062 - acc: 0.9698
Epoch 67/100
 - 1s - loss: 0.0986 - acc: 0.9698
Epoch 68/100
 - 1s - loss: 0.0982 - acc: 0.9687
Epoch 69/100
 - 1s - loss: 0.0954 - acc: 0.9776
Epoch 70/100
 - 1s - loss: 0.0935 - acc: 0.9720
Epoch 71/100
 - 1s - loss: 0.0944 - acc: 0.9720
Epoch 72/100
 - 1s - loss: 0.1062 - acc: 0.9676
Epoch 73/100
 - 1s - loss: 0.1020 - acc: 0.9653
Epoch 74/100
 - 1s - loss: 0.0969 - acc: 0.9732
Epoch 75/100
 - 1s - loss: 0.1076 - acc: 0.9597
Epoch 76/100
 - 1s - loss: 0.1415 - acc: 0.9564
Epoch 77/100
 - 1s - loss: 0.1113 - acc:

ResourceExhaustedError: OOM when allocating tensor with shape[39733,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_9/Adadelta/Variable_18/Assign = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_9/Adadelta/Variable_18, training_9/Adadelta/zeros)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_9/Adadelta/Variable_18/Assign', defined at:
  File "/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py"

In [ ]:
t = grid_result.best_estimator_

In [ ]:
print(t.score(X_test, y_test))

In [ ]:
# t.model.save('../model_gs/keras_fs.h5')

In [ ]:
# tf.set_random_seed(42)

# # make keras model
# start = time.time()
# inputs = Input(shape=(X_train.shape[1],))

# x = Dense(512, activation='relu')(inputs)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# x = Dense(512, activation='relu')(x)
# x = BatchNormalization()(x)
# x = Dropout(0.5)(x)

# predictions = Dense(
#     len(df['Subclass'].value_counts()), activation='softmax')(x)
# model = Model(inputs=inputs, outputs=predictions)

# # compile
# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adamax(),
#     metrics=['accuracy']
# )

# epochs = 100
# batch_size = 1000
# es = EarlyStopping(monitor='val_loss', patience=20)

# history = model.fit(
#     X_train,
#     y_train_for_keras,
#     epochs=epochs,
#     batch_size=batch_size,
#     validation_data=(X_test, y_test_for_keras),
#     verbose=1,
#     callbacks=[
#         es,
#     ]
# )

# model.save('../model/Keras_fs_pos.h5')

In [ ]:
load_model('../model/Keras_fs_pos.h5').evaluate(X_test, y_test_for_keras, verbose=0)[1]